In [1]:
import ray.rllib as rllib
from ray import tune
import ray

from ray.tune.registry import register_env

import gym

from baselines.common.atari_wrappers import make_atari, wrap_deepmind
from ray.rllib.utils.exploration.epsilon_greedy import EpsilonGreedy
from ray.rllib.utils.framework import try_import_tf

from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.misc import normc_initializer

from ray.rllib.algorithms.dqn.dqn import DQNConfig
from ray.rllib.algorithms.dqn import DQN 
from ray.rllib.algorithms.dqn.distributional_q_tf_model import DistributionalQTFModel


# from ray.rllib.env.wrappers.atari_wrappers import 

tf1, tf, tfv = try_import_tf()

# ray.init()

/home/deonix/miniconda3/envs/rllib/lib/python3.8/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/deonix/miniconda3/envs/rllib/lib/python3.8/site-packages/keras/utils/image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/home/deonix/miniconda3/envs/rllib/lib/python3.8/site-packages/keras/utils/image_utils.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/home/deonix/miniconda3/envs/rllib/lib/python3.8/site-packages/keras/utils/image_utils.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/home/deonix/

In [2]:
# def atari_env(env_config: dict):
#     render_mode = env_config.get("render_mode", None)
#     env = gym.make(env_config["name"], render_mode=render_mode)
    
# #     env = wrap_deepmind(env, frame_stack=True, scale=False)
#     return env

# register_env("atari_env", atari_env)


In [3]:
# https://github.com/ray-project/ray/blob/master/rllib/examples/custom_keras_model.py
# Eigendefiniertes Q_Model
class AtariModel(DistributionalQTFModel):
    def __init__(self,
                obs_space,
                action_space,
                num_outputs,
                model_config,
                name,
                **kw):
        
        super(AtariModel, self).__init__(
            obs_space, 
            action_space, 
            num_outputs, 
            model_config, 
            name, 
            **kw
        )
#         print(obs_space)
        self.inputs = tf.keras.layers.Input(shape=obs_space.shape,
                                           name="input_layer")
        layer1 = tf.keras.layers.Conv2D(32, 8, strides=4,
                                       name="layer1",
                                       activation=tf.nn.relu,
                                       kernel_initializer=normc_initializer(1.0)
                                       )(self.inputs)
        layer2 = tf.keras.layers.Conv2D(64, 4, strides=2,
                                       name="layer2",
                                       activation=tf.nn.relu,
                                       kernel_initializer=normc_initializer(1.0)
                                       )(layer1)
        layer3 = tf.keras.layers.Conv2D(64, 3, strides=1,
                                       name="layer3",
                                       activation=tf.nn.relu,
                                       kernel_initializer=normc_initializer(1.0)
                                       )(layer2)
        flatten = tf.keras.layers.Flatten()(layer3)
#         layer_4 = tf.keras.layers.Dense(
#                                         512,
#                                         activation=tf.nn.relu,
#                                         name="dense1",
#                                         kernel_initializer=normc_initializer(1.0)
#                                         )(flatten)
        layer_out = tf.keras.layers.Dense(
                                        num_outputs,
                                        name="my_out",
                                        activation=None,
                                        kernel_initializer=normc_initializer(0.01),
                                        )(flatten)

        self.base_model = tf.keras.Model(self.inputs, layer_out)
        
    def forward(self, input_dict, state, seq_lens):
        model_out = self.base_model(input_dict["obs"])
        
        return model_out, state
    
#     def metrics(self):
#         return {"foo": tf.constant(42.0)}


ModelCatalog.register_custom_model("AtariModelv2", AtariModel)

In [4]:
#Generelle Einstellungen
config = DQNConfig().to_dict()

DQNConfigTrainer = {
    #DQN
    "dueling": True,
    "double_q": True,
    "target_network_update_freq": 8000,
    "num_gpus": 1,
#     "num_workers": 3,
#     "num_envs_per_worker": 2,
    "rollout_fragment_length": 4, #4 = num_envs_per_worker * rollout_fragment_length
    #Einstellungen für den Algorithmus
    "env": "BreakoutNoFrameskip-v4",
#     "env_config" : {
#         "name": "BreakoutNoFrameskip-v4"
#     },
    "noisy": False,
    "n_step": 1,
    "num_atoms": 1,
    "gamma": 0.99,
    "adam_epsilon": 0.00015,
    "lr": 0.0000625, #Lernrate des TD-Errors
    "train_batch_size": 32,
    "hiddens": [512],
    "model": {
        "custom_model": "AtariModelv2",
        
    },
#     "clip_rewards": False, #Clippen zu -1, 0 und 1
#     "preprocessor_pref": "deepmind",
    #Exploration
    "explore": True,
    "exploration_config": {
        "type": "EpsilonGreedy",
        "epsilon_timesteps": 200000,
        "final_epsilon": 0.01
    },
#     "evaluation_interval": int(5e4),
#     "evaluation_num_workers": 1,
#     "evaluation_config": {
#         "num_envs_per_worker": 1,
#         "explore": False,
#         "render_env": True,
#         "env_config": {
#             "name": "BreakoutNoFrameskip-v4",
#             "render_mode": "rgb_array",
#             "record_env": True
#         }
#         "record_env": True
#     },
    #Kompression
    "compress_observations": True,
    #Replay Buffer
    "replay_buffer_config": {
            "type": "MultiAgentPrioritizedReplayBuffer",
            # Specify prioritized replay by supplying a buffer type that supports
            # prioritization, for example: MultiAgentPrioritizedReplayBuffer.
#             "prioritized_replay": DEPRECATED_VALUE,
            # Size of the replay buffer. Note that if async_updates is set,
            # then each worker will have a replay buffer of this size.
            "capacity": 200000,
            "prioritized_replay_alpha": 0.6,
            # Beta parameter for sampling from prioritized replay buffer.
            "prioritized_replay_beta": 0.4,
            # Epsilon to add to the TD errors when updating priorities.
            "prioritized_replay_eps": 1e-6,
            # The number of continuous environment steps to replay at once. This may
            # be set to greater than 1 to support recurrent models.
            "replay_sequence_length": 1,
            # Whether to compute priorities on workers.
            "worker_side_prioritization": False,
        },
}

config.update(DQNConfigTrainer)


In [ ]:
analysis = tune.run("DQN",
                    name="BreakoutDQNruntest",
#                     stop={"episode_reward_mean": 40},
                    config=config,
                    checkpoint_score_attr="episode_reward_mean",
                    keep_checkpoints_num=5,
                    checkpoint_freq=10,
                    checkpoint_at_end=True,
#                     resume="AUTO",
                   )

In [ ]:
analysis = tune.ExperimentAnalysis( 
    experiment_checkpoint_path="~/ray_results/BreakoutDQNrun2/experiment_state-2022-09-06_08-34-31.json")

In [ ]:
analysis.default_mode = "max"
analysis.default_metric = "episode_reward_mean"
analysis.best_trial
best_check = analysis.get_best_checkpoint(trial=analysis.get_best_trial())

In [ ]:
dqn_agent = DQN(config=config)
dqn_agent.restore(best_check)

In [ ]:
dqn_agent.evaluate()

In [ ]:
x = AlgorithmConfig().to_dict()

In [ ]:
x.get("env")


In [ ]:
env = atari_env({"name": "BreakoutNoFrameskip-v4", "render_mode": "human"})

In [ ]:
a = 1
b = 2
c = 3
a = b = a

In [ ]:
policy = dqn_agent.get_policy()
model = policy.model
model.base_model.summary() 
model.q_value_head.summary()
model.state_value_head.summary()

In [ ]:
print(config)